In [1]:
import sys
sys.path.append('/content/drive/MyDrive/MJ-Playground/AML-CW-1')

In [ ]:
pip install pytorch_lightning

In [12]:
from utils.utils import *
from algorithms.cnn import LitCNN

import pytorch_lightning as pl
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split

In [6]:
# seed fixation
pl.seed_everything(1)

Global seed set to 1


1

In [7]:
# path for colab only
import os
colab = '/content/drive/MyDrive/MJ-Playground/AML-CW-1/data'
PATH1 = os.path.join(colab, 'cifar-10-batches-py/data_batch_1')
PATH2 = os.path.join(colab, 'cifar-100-python/train')

# Load data
cifar10 = unpickle(PATH1)
cifar100 = unpickle(PATH2)

# Generate dataset
cifar10 = dataset_generator(cifar10, 3000, cifar_type='cifar10')
cifar100 = dataset_generator(cifar100, 3000, cifar_type='cifar100')

# test path
PATH1_t = os.path.join(colab, 'cifar-10-batches-py/data_batch_2')
PATH2_t = os.path.join(colab, 'cifar-100-python/test')

# Load test data
test_cifar10 = unpickle(PATH1_t)
test_cifar100 = unpickle(PATH2_t)

# Generate test dataset
test_cifar10 = dataset_generator(test_cifar10, 300, cifar_type='cifar10')
test_cifar100 = dataset_generator(test_cifar100, 300, cifar_type='cifar100')

# Normalise
cifar10['data'] = cifar10['data']/255
cifar100['data'] = cifar100['data']/255

test_cifar10['data'] = test_cifar10['data']/255
test_cifar100['data'] = test_cifar100['data']/255

# Reshape for CNN
n_samples, _ = cifar10['data'].shape
cifar10['data'] = cifar10['data'].reshape((n_samples, 3, 32, 32))
print('cifar10 shape: ', cifar10['data'].shape)

n_samples, _ = cifar100['data'].shape
cifar100['data'] = cifar100['data'].reshape((n_samples, 3, 32, 32))
print('cifar100 shape: ', cifar100['data'].shape)

n_samples, _ = test_cifar10['data'].shape
test_cifar10['data'] = test_cifar10['data'].reshape((n_samples, 3, 32, 32))
print('test_cifar10 shape: ', test_cifar10['data'].shape)

n_samples, _ = test_cifar100['data'].shape
test_cifar100['data'] = test_cifar100['data'].reshape((n_samples, 3, 32, 32))
print('test_cifar100 shape: ', test_cifar100['data'].shape)

cifar10 shape:  (3000, 3, 32, 32)
cifar100 shape:  (3000, 3, 32, 32)
test_cifar10 shape:  (300, 3, 32, 32)
test_cifar100 shape:  (300, 3, 32, 32)


In [8]:
# batch size
BATCH_SIZE = 16
# GPU
AVAIL_GPUS = min(1, torch.cuda.device_count())

In [9]:
# cifar10 dataloader
X = torch.tensor(cifar10['data'], dtype=torch.float32)
Y = torch.tensor(cifar10['labels'], dtype=torch.int64)

cifar10_dataset = TensorDataset(X, Y)
cifar10_train_loader = DataLoader(cifar10_dataset, batch_size = BATCH_SIZE)

X = torch.tensor(test_cifar10['data'], dtype=torch.float32)
Y = torch.tensor(test_cifar10['labels'], dtype=torch.int64)

test_cifar10_dataset = TensorDataset(X, Y)
test_cifar10_train_loader = DataLoader(test_cifar10_dataset, batch_size = BATCH_SIZE)

# cifar100 dataloader
X = torch.tensor(cifar100['data'], dtype=torch.float32)
Y = torch.tensor(cifar100['labels'], dtype=torch.int64)

cifar100_dataset = TensorDataset(X, Y)
cifar100_train_loader = DataLoader(cifar100_dataset, batch_size = BATCH_SIZE)

X = torch.tensor(test_cifar100['data'], dtype=torch.float32)
Y = torch.tensor(test_cifar100['labels'], dtype=torch.int64)

test_cifar100_dataset = TensorDataset(X, Y)
test_cifar100_train_loader = DataLoader(test_cifar100_dataset, batch_size = BATCH_SIZE)

### cifar10

In [10]:
cnn_model = LitCNN()

trainer = pl.Trainer(
    gpus=AVAIL_GPUS,
    max_epochs=50,
    progress_bar_refresh_rate=2,)

import time
start = time.time()

trainer.fit(model=cnn_model, train_dataloaders=cifar10_train_loader)

end = time.time()
print('training time: ', round(end-start,1),end='')

start = time.time()

trainer.test(test_dataloaders=test_cifar10_train_loader)

end = time.time()
print('inference time: ', round(end-start,1),end='')

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=2)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Missing logger folder: /content/lightning_logs

  | Name     | Type       | Params
----------------------------------------
0 | model    | Sequential | 62.0 K
1 | accuracy | Accuracy   | 0     
----------------------------------------
62.0 K    Trainable params
0         Non-trainable para

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:907: LightningDeprecationWarning: `trainer.test(test_dataloaders)` is deprecated in v1.4 and will be removed in v1.6. Use `trainer.test(dataloaders)` instead.
  "`trainer.test(test_dataloaders)` is deprecated in v1.4 and will be removed in v1.6."
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1399: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `test(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  f"`.{fn}(ckpt_path=None)` was called without a model."
Restoring states from the checkpoint path at /content/lightning_logs/version_0/checkpoints/epoch=49-step=9399.ckpt


training time:  88.1

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /content/lightning_logs/version_0/checkpoints/epoch=49-step=9399.ckpt


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.4533333480358124, 'test_loss': 1.6884366273880005}
--------------------------------------------------------------------------------
inference time:  2.2

### cifar100

In [10]:
cnn_model = LitCNN(num_classes=100)

trainer = pl.Trainer(
    gpus=AVAIL_GPUS,
    max_epochs=50,
    progress_bar_refresh_rate=2,)

import time
start = time.time()

trainer.fit(model=cnn_model, train_dataloaders=cifar100_train_loader)

end = time.time()
print('training time: ', round(end-start,1),end='')

start = time.time()

trainer.test(test_dataloaders=test_cifar100_train_loader)

end = time.time()
print('inference time: ', round(end-start,1),end='')

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=2)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Missing logger folder: /content/lightning_logs

  | Name     | Type       | Params
----------------------------------------
0 | model    | Sequential | 69.7 K
1 | accuracy | Accuracy   | 0     
----------------------------------------
69.7 K    Trainable params
0         Non-trainable para

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:907: LightningDeprecationWarning: `trainer.test(test_dataloaders)` is deprecated in v1.4 and will be removed in v1.6. Use `trainer.test(dataloaders)` instead.
  "`trainer.test(test_dataloaders)` is deprecated in v1.4 and will be removed in v1.6."
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1399: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `test(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  f"`.{fn}(ckpt_path=None)` was called without a model."
Restoring states from the checkpoint path at /content/lightning_logs/version_0/checkpoints/epoch=49-step=9399.ckpt


training time:  84.2

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /content/lightning_logs/version_0/checkpoints/epoch=49-step=9399.ckpt


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.15666666626930237, 'test_loss': 3.9208576679229736}
--------------------------------------------------------------------------------
inference time:  2.1